# Pair Programming



**Learning Outcomes**

<li> Describe how the oracle can be applied to a pair of candidate solutions to determine if the secret combination is present.

<li> Determine the average number of queries required to find a solution when testing in pairs.

Author: [Monit Sharma](https://github.com/MonitSharma)
LinkedIn: [Monit Sharma](https://www.linkedin.com/in/monitsharma/)
Twitter: [@MonitSharma1729](https://twitter.com/MonitSharma1729)
Medium : [MonitSharma](https://medium.com/@_monitsharma)

Applying the oracle by itself is not enough to improve the lock picker. The problem is that we introduce a phase change which is unobservable without further processing.

---

In order to make some progress, we must combine states to induce a *relative* change of phase which is detectable. 

-----

Let's take a superposition of two states $|x⟩$ and $|y⟩$ , and apply the oracle.

$$|\psi_{xy}⟩ = \frac{1}{\sqrt{2}} (|x⟩ + |y⟩)$$

$$ \frac{1}{\sqrt{2}} \bigg((-1)^{f(x) }|x⟩ + (-1)^{f(y)} |y⟩ \bigg)$$



Instead of working in the space spanned by $|x⟩, |y⟩$, we could measure in the orthonormal basis $\frac{1}{\sqrt{2}}(|x⟩ + |y⟩) , \frac{1}{\sqrt{2}}(|x⟩ - |y⟩)$. 

----

If we measure and observe the second state, we know one of the states had its phase flipped, so the correct combination is either $x$ or $y$. But if we get $+$ then no phase got flipped and neither $x$ and $y$ is the right combination. We give an illustration below, where $t_1, ..., t_7$ stand for non-solutions and $s$ is the secret combination:


![](https://codebook.xanadu.ai/pics/pairs.svg)

This leads to rudimentary algorithm: test solutions in pairs. This is almost the brute force algorithm except that instead of searching through $2^n$ combinations, we search through $2^{n-1}$ pairs, so we improve by a constant factor. Once we have identified the correct pair, we can just test both classically.

----

This last step doesn't scale with $n$ so we have a very modest quantum speedup!. In principle we could divide the full set of strings $\{0,1\}^n$ into pairs any way we please. A particularly simple scheme is to split each $n$ but string $x \in \{0,1\}^n$ into first $n-1$ bits andits last bit.

$$ 00101010 \rightarrow (0010101,0) $$

------

Since there are two possible values for the last bit $0$ and $1$ we can label pairs by the first $n-1$ bits. For instance the $7$ bit strings $0010101$ labels the two $8$ bit strings

$$00101010 → 00101010, 00101011 $$

We'll label the basis states on $n-1$ qubits using the string $x$

------
------

Given this method for splitting all the strings into pair based on the first $(n-1)$ bits labelled by $x$ we can provide a fairly concrete scheme for finding the secret combination to our lock. Let $H_n$ refer to the Hadamard acting on the last qubit.

-----

Since $H|0⟩ = |+⟩$, if the $n$th qubit is in state $|0⟩$ acting with $H_n$ gives us $|+⟩$ on that qubit and hence $|\psi_x⟩ = H_n |x0⟩$.

Applying the oracle yields

$$ U_f|\psi_x⟩ = |x⟩ \otimes  \frac{1}{\sqrt{2}} \bigg((-1)^{f(x0) }|0⟩ + (-1)^{f(x1)} |1⟩ \bigg)$$


**Solution Present**
$$ \pm |x⟩ \otimes |-⟩ $$

**otherwise**

$$ |x⟩ \otimes |+⟩$$

----

If we apply $H_n$, once more we convert the $|+⟩$ state to $|0⟩$ and the state $|-⟩$ state to $|1⟩$

$$ |\phi⟩ = H_n U_f |\psi_x⟩ $$

**Solution Present**

$$\pm |x1⟩$$

**Otherwise**

$$ \pm |x0⟩$$

-----

So we can simply measure the last qubit and see what we get. 
Here's the circuit diagram


![](https://codebook.xanadu.ai/pics/flowchart.svg)

In [ ]:
%pip install pennylane

## Code Exercise

The oracle simply adds a phase to the solution 

$$|s⟩ → -|s⟩$$

This doesn't change the probability of observing $|s⟩$ , but we can observe a *relative* phase change if $|s⟩$ is combined with a non-solution $|t⟩$


$$|t⟩ + |s⟩ → |t⟩ - |s⟩$$

-----

Thus the oracle $U_f$ provides a way to test for the presence of the solution in a superposition of computational basis states.

### Codercise A.3.1

Implement this circuit and return the probabilities on the last qubit. The function `oracle_matrix` is defined for you. You can expand the box below to see the docstring and implementation.

![](https://codebook.xanadu.ai/pics/pair-test-circuit.svg)

In [3]:
import numpy as np
import pennylane as qml

In [4]:
def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.
    
    Args:
        combo (list[int]): A list of bits representing a secret combination.
         
    Returns: 
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2]*len(combo)) # Index of solution
    my_array = np.identity(2**len(combo)) # Create the identity matrix
    my_array[index, index] = -1
    return my_array

In [5]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """Test a pair labelled by x_tilde for the presence of a solution.
    
    Args:
        x_tilde (list[int]): An (n_bits - 1)-string labelling the pair to test.
        combo (list[int]): A secret combination of n_bits 0s and 1s.
        
    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)
    qml.Hadamard(wires=n_bits-1)        
    qml.QubitUnitary(oracle_matrix(combo), wires = [ i for i in range(n_bits)])
    qml.Hadamard(wires=n_bits-1)

    ##################
    # YOUR CODE HERE #
    ##################
    
    return qml.probs(wires=n_bits-1)


With this circuit at our disposal, we can crack the lock by simply iterating over the labels $x$  until we detect the solution. Let's see how long this takes on average.

### Codercise A.3.2

 Complete the function below to see how many attempts it takes to break the lock using our quantum circuit. You should find an improvement over the brute force approach, which takes around $9$ guesses (on average) for  qubits. Note that `pair_circuit` is available.

In [6]:
def pair_lock_picker(trials):
    """Create a combo, run pair_circuit until it succeeds, and tally success rate.
    
    Args:
        trials (int): Number of times to test the lock picker.

    Returns:
        float: The average number of times the lock picker uses pair_circuit.
    """
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs] 

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits) # Random list of bits
        counter = 0
        for x_tilde in x_tildes:
            counter += 1

            ##################
            # YOUR CODE HERE #
            ##################
            pair = pair_circuit(x_tilde,combo)
            if np.isclose(1,pair[1]):
                break
        
        test_numbers.append(counter)
    return sum(test_numbers)/trials

trials = 500
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")


For 4 bits, it takes 4.596 pair tests on average.